# Imports

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
%%capture

from djimaging.user.alpha.schemas.alpha_schema import *
from djimaging.user.alpha.utils import populate_alpha

populate_alpha.load_alpha_config(schema_name=populate_alpha.SCHEMA_PREFIX + "glu")
populate_alpha.load_alpha_schema(create_schema=False, create_tables=False)

In [ ]:
schema

# Draw templates

In [ ]:
# pres_key = {'experimenter': 'Ran', 'date': '20201103', 'exp_num': 2, 'field': 'd1', 'stim_name': 'noise_1500'}
pres_key = {'experimenter': 'Ran', 'date': '20201111', 'exp_num': 1, 'field': 'd4', 'stim_name': 'noise_1500'}

In [ ]:
from djimaging.utils import scanm_utils
from djimaging.utils import math_utils

fs = (Presentation.ScanInfo() & pres_key).fetch1('scan_frequency')
h5_header, triggertimes = (Presentation() & pres_key).fetch1('h5_header', 'triggertimes')
ch_stacks, wparams = scanm_utils.load_stacks_from_h5(h5_header)

In [ ]:
stack = ch_stacks['wDataCh1'].copy()
stack[0, :] = stack.mean()
stack.shape

In [ ]:
alt_stack = ch_stacks['wDataCh0'].copy()
alt_stack[0, :] = alt_stack.mean()
alt_stack.shape

In [ ]:
field_file = (Field() & pres_key).fetch1('fromfile')
field_file

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 3))
axs[0].imshow(np.mean(stack, axis=2).T)
axs[1].imshow(np.std(stack, axis=2).T)
plt.show()

In [ ]:
mean_map = np.mean(stack, axis=2)
std_map = np.std(stack, axis=2)
norm_map = mean_map / std_map
plt.imshow((norm_map > 2).T & (mean_map > np.percentile(mean_map, 90)).T);

In [ ]:
dsf = 8

dat = stack[3:, :, int(triggertimes[0] * fs):].copy()
dat = dat[:, :, :dsf * (dat.shape[2] // dsf)].reshape((dat.shape[0], dat.shape[1], dat.shape[2] // dsf, dsf))
dat = np.mean(dat, axis=-1)
dat = math_utils.normalize_zscore(dat)

In [ ]:
from ipywidgets import HTML
from alphacnn.visualize import plot_video

anim = plot_video.array_to_anim(
    dat.T, pos=None, fps=20,
    axis_off=False, xy_upsample=0, pixel_size=1, cbar=True, cmap_sym=True)
HTML(anim.to_html5_video())

In [ ]:
(Field() & pres_key).plot1()

# Draw

In [ ]:
from djimaging.autorois.roi_canvas import InteractiveRoiCanvas
import os

In [ ]:
gui = InteractiveRoiCanvas(ch0_stacks=[stack[:, :]], ch1_stacks=[alt_stack[:, :]], output_files=[None], upscale=3)

In [ ]:
gui.output_files = [os.path.split(field_file)[1].replace('.h5', '_match_template.pkl')]
gui.output_file = gui.output_files[0]

In [ ]:
gui.start_gui()